In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import earthpy as et
import seaborn as sns
import plotly.express as px
from shapely.geometry import Point, Polygon

In [2]:
#Function that checks if the coordinates of the radiation data are located inside the
#selected polygon.
def evaluate_point(polygon,global_radiation):
    points_inside_polygon = []
    for i in range(len(global_radiation)):
        point = Point(global_radiation.iloc[i,1],global_radiation.iloc[i,0])
        if point.within(polygon) == True:
            points_inside_polygon.append(point)
    
    return points_inside_polygon

In [3]:
#Function that gets the average radiation for each state.
def average_radiation(points,global_radiation):
    radiation_points = []
    average_radiation =[]
    for i in range(len(points)):
        for point in points[i]:
            LON = point.x
            LAT = point.y
            row = global_radiation.loc[(global_radiation['LON'] == LON) & (global_radiation['LAT'] == LAT)]
            radiation = row["ANN"]
            radiation_points.append(float(radiation))
        average_radiation.append(sum(radiation_points) / len(radiation_points))
    
    return average_radiation
            

In [4]:
#Function that computes the global radiation per month per state
def average_radiation_month(points,global_radiation):
    radiation_points = []
    average_radiation =[]
    radiation = []
    for i in range(len(points)):
        for point in points[i]:
            LON = point.x
            LAT = point.y
            row = global_radiation.loc[(global_radiation['LON'] == LON) & (global_radiation['LAT'] == LAT)]
            radiation = [row.iloc[0,rad] for rad in range(3,16)]         
            radiation_points.append((radiation))
            radiation_array = np.array(radiation_points)
        average_radiation.append(np.average(radiation_array,axis=0))
    
    return average_radiation

In [5]:
#Read the shp files.
mapa = gpd.read_file('México_Estados.shp')

In [6]:
#Read NASA's file that contains the global average radiations
global_radiation = pd.read_csv("POWER_Global_Climatology_8fd627aa_mexico.csv")

In [7]:
#Apply the average radiation per month funtion to all the polygons inside the "mapa" geopandas dataframe
radiation = []
state_list = []
for state in range(len(mapa)):
    state_list.append(str(mapa["ESTADO"][state]))
    points_inside_polygon = evaluate_point(mapa["geometry"][state],global_radiation)
    radiation.append(points_inside_polygon)
avg_radiations = average_radiation_month(radiation,global_radiation)

In [8]:
#Covert the list of lists with the average radiations to a dataframe and add the month name to the columns.
months = ["Ene","Feb","Mar","Abr","May","Jun","Jul","Ago","Sep","Oct","Nov","Dic","Anual"]
df_rad_avg_day = pd.DataFrame(avg_radiations,index=state_list,columns=months)

In [14]:
df_rad_avg_day.loc[:,'Anual'].mean()

5.582480024585347

In [9]:
df_rad_avg_day

,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic,Anual
Baja California,3.529310,4.352069,5.547241,6.668621,7.005862,6.980690,6.688276,6.209655,5.608621,4.633793,3.828966,3.265517,5.360690
Baja California Sur,3.751964,4.568929,5.766250,6.735893,7.124107,7.083929,6.740714,6.254821,5.653750,4.833036,4.031250,3.463393,5.500357
Nayarit,3.870769,4.710462,5.909846,6.825846,7.206000,7.050308,6.645538,6.202000,5.615692,4.919538,4.166615,3.584769,5.558615
Jalisco,4.146882,5.028280,6.204086,6.958710,7.202366,6.811290,6.378495,6.035054,5.497419,5.035806,4.455591,3.880215,5.635484
Aguascalientes,4.163438,5.047708,6.221875,6.963646,7.199792,6.798229,6.368333,6.033438,5.496563,5.048437,4.476771,3.900938,5.642500
Guanajuato,4.217290,5.109533,6.263738,6.957290,7.155514,6.742804,6.329533,6.025514,5.480187,5.063832,4.530280,3.962150,5.652523
Querétaro,4.230360,5.125766,6.271892,6.950090,7.137477,6.724144,6.316757,6.023063,5.470631,5.062342,4.540721,3.975135,5.651712
Hidalgo,4.242458,5.137797,6.265847,6.916356,7.092203,6.675085,6.283983,6.004831,5.440254,5.044153,4.543305,3.988644,5.635763
Michoacán,4.333529,5.240588,6.353971,6.943015,7.043750,6.556250,6.183382,5.927132,5.379191,5.044191,4.614265,4.080294,5.641029
México,4.364236,5.271528,6.368681,6.926528,7.007222,6.512569,6.160486,5.911111,5.364444,5.045556,4.633125,4.110069,5.638958


In [10]:
#Read the csv file with the area of each state
state_area = pd.read_csv('Areas_Estados.csv')
state_area.set_index('Estado',inplace=True)

In [11]:
state_area.head()

,Superficie km2,Superficie_m2
Estado,,
Aguascalientes,5616,5.616000e+09
Baja California,71450,7.145000e+10
Baja California Sur,73909,7.390900e+10
Campeche,57507,5.750700e+10
Chiapas,73311,7.331100e+10


In [12]:
# Compute the total radiation for each state by multiplying the average radiation per day (kW-hr/m^2/day)
# by the area of each state to obtain the total radiation per state per day (kW-hr/day), then multiplying that quantity
# by the number of days in each month to obtain the total radiation per month and annually.(kW-hr)
# The final step is to divide the value by 1000M to obtain the value in TW-hr.
total_radiation = df_rad_avg_day.copy()
days_month = {'Ene': 31, 'Feb': 28, 'Mar': 31, 'Abr': 30, 'May': 31, 'Jun': 30,
              'Jul': 31, 'Ago': 31, 'Sep': 30, 'Oct': 31, 'Nov': 30, 'Dic': 31, 'Anual': 365}
for state in state_area.index:
    for key,value in days_month.items():
            total_radiation.loc[state][key] = (total_radiation.loc[state][key] * state_area.loc[state][1] * value) / 1000000000

total_radiation.head()

,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic,Anual
Baja California,7817.245948,8706.749172,12286.862293,14294.188448,15517.634190,14963.108276,14814.196621,13754.075724,12022.078448,10263.620034,8207.387586,7232.957414,139802.765690
Baja California Sur,8596.421780,9455.178370,13211.510909,14935.293155,16322.604679,15706.982304,15444.183016,14330.915506,12535.890263,11073.349935,8938.369687,7935.252983,148381.952066
Nayarit,3342.668572,3674.141158,5103.548114,5704.427889,6222.863802,5892.012642,5738.867713,5355.842534,4693.090218,4248.351071,3482.082143,3095.688410,56518.917301
Jalisco,10102.749360,11064.548175,15114.568080,16406.132284,17546.604720,16058.570516,15539.467200,14702.766960,12960.935768,12268.372680,10504.680503,9453.088560,161651.713355
Aguascalientes,724.837815,793.742040,1083.203550,1173.235050,1253.454930,1145.365650,1108.701360,1050.397335,926.060850,878.912775,754.246350,679.137615,11566.222200


In [13]:
# Reading the monthly energy consumption file.
yearly_consumption = pd.read_csv('energy_consumption_2018_2020.csv')

In [14]:
# Melting data frame to graph yearly energy consumption vs solar radiation in scatter plot
yearly_consumption = yearly_consumption.melt(id_vars=['Estado'],var_name='Year',value_name='Consumption')
yearly_consumption.set_index('Estado',inplace=True)
yearly_consumption.drop('Total',inplace=True)

In [15]:
# Adding yearly average radiation to all rows
yearly_consumption = yearly_consumption.assign(Radiation= lambda x: round(total_radiation.loc[x.index]['Anual'],2))

In [16]:
yearly_consumption.head()

,Year,Consumption,Radiation
Estado,,,
Aguascalientes,2018,2855500,11566.22
Baja California,2018,10823800,139802.77
Baja California Sur,2018,2561200,148381.95
Campeche,2018,1315600,117967.63
Chiapas,2018,3096400,148325.68


In [23]:
#Plotting interactive scatter plot. Click "Autoscale" on the plot menu after plotting to see the graph.
fig = px.scatter(yearly_consumption, x='Consumption', y="Radiation", animation_frame="Year", 
                 animation_group=yearly_consumption.index, size="Consumption", 
                 color=yearly_consumption.index, hover_name=yearly_consumption.index,
                 log_x=True, size_max=55, range_x=[100,100000], range_y=[25,90],
                labels=dict(Consumption='Consumption kWhr/year', Radiation="Radiation TWhr/year"))
fig.show()